In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [3]:
%%sql
CREATE TABLE Library (
    library_name TEXT,
    address TEXT, 
    phone_number TEXT,
    email TEXT,
    PRIMARY KEY (library_name, address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [4]:
%%sql
CREATE TABLE Item (
    item_id INTEGER PRIMARY KEY,
    category_id INTEGER,
    library_name TEXT,
    address TEXT,
    title TEXT,
    status TEXT,
    genre TEXT,
    location TEXT,
    publisher_name TEXT,
    FOREIGN KEY (category_id)
        REFERENCES ItemCategory (category_id)
    FOREIGN KEY (library_name)
        REFERENCES Library (library_name),
    FOREIGN KEY (address) 
        REFERENCES Library (address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [5]:
%%sql
CREATE TABLE ItemCategory (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [6]:
%%sql
CREATE TABLE Reading (
    item_id INTEGER PRIMARY KEY,
    isbn INTEGER,
    author TEXT,
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [7]:
%%sql
CREATE TABLE Music (
    item_id INTEGER PRIMARY KEY,
    artist TEXT,
    num_songs INTEGER,
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [8]:
%%sql
CREATE TABLE User (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    birthdate DATE NOT NULL,
    phone_number TEXT NOT NULL,
    address TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    member_since DATE DEFAULT (datetime('now', 'localtime')),
    favourite_genre TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [10]:
%%sql
CREATE TABLE Employee (
    user_id INTEGER PRIMARY KEY,
    job_title TEXT,
    salary INTEGER,
    FOREIGN KEY (user_id)
        REFERENCES User (user_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [11]:
%%sql
CREATE TABLE Events (
    event_id INTEGER PRIMARY KEY,
    library_name TEXT,
    address TEXT,
    room_number INTEGER,
    description TEXT,
    event_date DATE,
    start_time TEXT,
    end_time TEXT,
    FOREIGN KEY (library_name, address)
        REFERENCES Library (library_name, address)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [12]:
%%sql
CREATE TABLE Audience (
    audience_id INTEGER PRIMARY KEY,
    type TEXT,
    genre TEXT
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [ ]:
%%sql
CREATE TABLE Borrows (
    user_id INTEGER,
    item_id INTEGER,
    borrow_date DATE DEFAULT (datetime('now', 'localtime')),
    due_date DATE DEFAULT (datetime('now', '+14 days', 'localtime')),
    return_date DATE,
    fine INTEGER DEFAULT 20,
    PRIMARY KEY (user_id, item_id, borrow_date),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (item_id)
        REFERENCES Item (item_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [ ]:
%%sql
DROP TABLE Borrows;

: 

In [14]:
%%sql
CREATE TABLE Attending (
    user_id INTEGER,
    event_id INTEGER,
    PRIMARY KEY (user_id, event_id),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (event_id)
        REFERENCES Events (event_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [15]:
%%sql
CREATE TABLE RecommendedFor (
    event_id INTEGER,
    audience_id INTEGER,
    PRIMARY KEY (event_id, audience_id),
    FOREIGN KEY (event_id)
        REFERENCES Events (event_id),
    FOREIGN KEY (audience_id)
        REFERENCES Audience (audience_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [16]:
%%sql
CREATE TABLE BelongsTo (
    user_id INTEGER,
    audience_id INTEGER,
    PRIMARY KEY (user_id, audience_id),
    FOREIGN KEY (user_id)
        REFERENCES User (user_id),
    FOREIGN KEY (audience_id)
        REFERENCES Audience (audience_id)
);

Running query in 'sqlite:///library.db'

++
||
++
++

In [17]:
%%sql
CREATE TRIGGER UpdateFavGenre AFTER INSERT ON Borrows
FOR EACH ROW
BEGIN
    UPDATE User
    SET favourite_genre = (
        SELECT genre
        FROM Item
        JOIN Borrows ON Item.item_id = Borrows.item_id
        WHERE Borrows.user_id = NEW.user_id
        GROUP BY Item.genre
        ORDER BY COUNT(*) DESC, MAX(Borrows.borrow_date) DESC
        LIMIT 1
    ) WHERE user_id = NEW.user_id;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [36]:
%%sql
DROP TRIGGER IF EXISTS UpdateUserAudience;

Running query in 'sqlite:///library.db'

++
||
++
++

In [37]:
%%sql
/*
    Okay to explain this so I dont seem insane:
    1. Upon update of favourite_genre, we need to first calculate the users age to ensure we have their most updated age range
    2. Then, we need to use this Age to determine which "type" of audience they fall into (child, teen, adult). 
    3. Then, we need to check if the combination of "type" and "genre" exists within the entity Audience.
    4. If it does not ecist, we need to insert it into Audience - this allows dynamic adding of Audiences.
    5. Then we need to grab either the newly created, or existing audience's id, and update the BelongTo table with the user_id and audience_id.
*/
CREATE TRIGGER UpdateUserAudience
AFTER UPDATE OF favourite_genre ON User
FOR EACH ROW
WHEN NEW.favourite_genre IS NOT NULL
BEGIN
    DELETE FROM BelongsTo
    WHERE user_id = NEW.user_id;

    INSERT OR IGNORE INTO Audience (type, genre)
    SELECT
        CASE
            WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
            WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
            ELSE 'Adults'
        END AS audience_type,
        NEW.favourite_genre
    FROM User
    WHERE user_id = NEW.user_id
    AND NOT EXISTS (
        SELECT 1
        FROM Audience
        WHERE
            type = CASE
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
                ELSE 'Adults'
            END
            AND genre = NEW.favourite_genre
    );

    -- Update the BelongsTo table with the user_id and audience_id
    INSERT OR REPLACE INTO BelongsTo (user_id, audience_id)
    SELECT
        NEW.user_id,
        Audience.audience_id
    FROM Audience
    WHERE
        type = (
            SELECT CASE
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) < 13 THEN 'Children'
                WHEN CAST((julianday('now') - julianday(birthdate)) / 365.2422 AS INTEGER) BETWEEN 13 AND 19 THEN 'Teens'
                ELSE 'Adults'
            END
            FROM User
            WHERE user_id = NEW.user_id
        )
        AND genre = NEW.favourite_genre;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

# Sample data being generated

In [70]:
%%sql
INSERT INTO Library (library_name, address, phone_number, email) VALUES
('Burnaby Public Library', '7311 Kingsway', '604-522-3971', 'info@burnabylibrary.ca');

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [73]:
%%sql
INSERT INTO ItemCategory (category_id, category_name) VALUES
(1, 'Book'),
(2, 'Journal'),
(3, 'Online Book'),
(4, 'Magazine'),
(5, 'CD'),
(6, 'Vinyl');

Running query in 'sqlite:///library.db'

6 rows affected.

++
||
++
++

In [74]:
%%sql
INSERT INTO Item (item_id, category_id, library_name, address, title, status, genre, location, publisher_name) VALUES
(1, 1, 'Burnaby Public Library', '7311 Kingsway', 'The Great Gatsby', 'Available', 'Fiction', 'A1', 'Scribner'),          -- Book
(2, 1, 'Burnaby Public Library', '7311 Kingsway', 'Dune', 'Borrowed', 'Science Fiction', 'B2', 'Chilton Books'),         -- Book
(3, 1, 'Burnaby Public Library', '7311 Kingsway', 'Gone Girl', 'Available', 'Mystery', 'C3', 'Crown Publishing'),        -- Book
(4, 1, 'Burnaby Public Library', '7311 Kingsway', 'Pride and Prejudice', 'Available', 'Romance', 'D4', 'T. Egerton'),   -- Book
(5, 1, 'Burnaby Public Library', '7311 Kingsway', 'Steve Jobs', 'Borrowed', 'Biography', 'E5', 'Simon & Schuster'),      -- Book
(6, 5, 'Burnaby Public Library', '7311 Kingsway', 'Abbey Road', 'Available', 'Rock', 'F6', 'Apple Records'),             -- CD
(7, 5, 'Burnaby Public Library', '7311 Kingsway', 'Thriller', 'Available', 'Pop', 'G7', 'Epic Records'),                 -- CD
(8, 6, 'Burnaby Public Library', '7311 Kingsway', 'A Night at the Opera', 'Available', 'Rock', 'H8', 'EMI Records'),     -- Vinyl
(9, 6, 'Burnaby Public Library', '7311 Kingsway', 'Led Zeppelin IV', 'Available', 'Rock', 'I9', 'Atlantic Records'),     -- Vinyl
(10, 5, 'Burnaby Public Library', '7311 Kingsway', 'The Dark Side of the Moon', 'Available', 'Progressive Rock', 'J10', 'Harvest Records'); -- CD

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [75]:
%%sql
INSERT INTO Reading (item_id, isbn, author) VALUES
(1, 9780743273565, 'F. Scott Fitzgerald'), -- The Great Gatsby
(2, 9780441172719, 'Frank Herbert'),      -- Dune
(3, 9780307588364, 'Gillian Flynn'),      -- Gone Girl
(4, 9780141439518, 'Jane Austen'),        -- Pride and Prejudice
(5, 9781451648539, 'Walter Isaacson');    -- Steve Jobs

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [76]:
%%sql
INSERT INTO Music (item_id, artist, num_songs) VALUES
(6, 'The Beatles', 17),       -- Abbey Road
(7, 'Michael Jackson', 9),    -- Thriller
(8, 'Queen', 12),             -- A Night at the Opera
(9, 'Led Zeppelin', 8),       -- Led Zeppelin IV
(10, 'Pink Floyd', 10);       -- The Dark Side of the Moon

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [77]:
%%sql
INSERT INTO User (user_id, first_name, last_name, birthdate, phone_number, address, email, favourite_genre) VALUES
(1, 'John', 'Doe', '1980-05-15', '604-111-1111', '123 Elm St, Burnaby', 'john.doe@email.com', NULL),
(2, 'Jane', 'Smith', '1990-08-22', '604-222-2222', '456 Oak St, Burnaby', 'jane.smith@email.com', NULL),
(3, 'Alice', 'Johnson', '2005-03-10', '604-333-3333', '789 Pine St, Burnaby', 'alice.johnson@email.com', NULL),
(4, 'Bob', 'Brown', '2008-07-12', '604-444-4444', '101 Maple St, Burnaby', 'bob.brown@email.com', NULL),
(5, 'Charlie', 'Davis', '1995-03-25', '604-555-5555', '202 Cedar St, Burnaby', 'charlie.davis@email.com', NULL),
(6, 'Diana', 'Evans', '2015-09-18', '604-666-6666', '303 Birch St, Burnaby', 'diana.evans@email.com', NULL),
(7, 'Eve', 'Garcia', '1982-12-05', '604-777-7777', '404 Spruce St, Burnaby', 'eve.garcia@email.com', NULL),
(8, 'Frank', 'Harris', '1978-06-30', '604-888-8888', '505 Fir St, Burnaby', 'frank.harris@email.com', NULL),
(9, 'Grace', 'Lee', '2003-04-07', '604-999-9999', '606 Willow St, Burnaby', 'grace.lee@email.com', NULL),
(10, 'Henry', 'Martinez', '2010-02-14', '604-000-0000', '707 Ash St, Burnaby', 'henry.martinez@email.com', NULL);

Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [78]:
%%sql
INSERT INTO Events (event_id, library_name, address, room_number, description, event_date, start_time, end_time) VALUES
(1, 'Burnaby Public Library', '7311 Kingsway', 101, 'Book Club: Fantasy Novels', '2025-03-21', '18:00', '20:00'),
(2, 'Burnaby Public Library', '7311 Kingsway', 102, 'Teen Mystery Night', '2025-03-22', '19:00', '21:00'),
(3, 'Burnaby Public Library', '7311 Kingsway', 103, 'Children’s Storytime', '2025-03-23', '10:00', '11:00'),
(4, 'Burnaby Public Library', '7311 Kingsway', 104, 'Romance Author Talk', '2025-03-24', '18:30', '20:00'),
(5, 'Burnaby Public Library', '7311 Kingsway', 105, 'Horror Movie Night', '2025-03-25', '19:00', '22:00'),
(6, 'Burnaby Public Library', '7311 Kingsway', 106, 'Sci-Fi Book Discussion', '2025-03-26', '19:00', '21:00'),
(7, 'Burnaby Public Library', '7311 Kingsway', 107, 'Historical Fiction Workshop', '2025-03-27', '14:00', '16:00'),
(8, 'Burnaby Public Library', '7311 Kingsway', 108, 'Poetry Reading: Modern Classics', '2025-03-28', '18:00', '20:00'),
(9, 'Burnaby Public Library', '7311 Kingsway', 109, 'Craft Workshop: DIY Bookmarks', '2025-03-29', '15:00', '17:00'),
(10, 'Burnaby Public Library', '7311 Kingsway', 110, 'Music Performance: Local Band', '2025-03-30', '19:00', '21:00'),
(11, 'Burnaby Public Library', '7311 Kingsway', 101, 'Book Club: Mystery Novels', '2025-03-31', '18:00', '20:00'),
(12, 'Burnaby Public Library', '7311 Kingsway', 102, 'Teen Coding Workshop', '2025-04-01', '19:00', '21:00'),
(13, 'Burnaby Public Library', '7311 Kingsway', 103, 'Children’s Art Class', '2025-04-02', '10:00', '11:00'),
(14, 'Burnaby Public Library', '7311 Kingsway', 104, 'Romantic Poetry Reading', '2025-04-03', '18:30', '20:00'),
(15, 'Burnaby Public Library', '7311 Kingsway', 105, 'Classic Horror Movie Night', '2025-04-04', '19:00', '22:00');

Running query in 'sqlite:///library.db'

15 rows affected.

++
||
++
++

In [79]:
%%sql 
-- User 1 borrows items
INSERT INTO Borrows (user_id, item_id, borrow_date, due_date, return_date, fine) VALUES
(1, 1, '2023-10-01', '2023-10-15', '2023-10-14', NULL),  -- Returned on time
(1, 2, '2023-10-05', '2023-10-19', NULL, NULL),          -- Still borrowed
(1, 6, '2023-10-10', '2023-10-24', '2023-10-25', 5),     -- Returned late, $5 fine
(1, 7, '2023-10-12', '2023-10-26', NULL, NULL),          -- Still borrowed
(1, 9, '2023-10-15', '2023-10-29', '2023-10-28', NULL);  -- Returned on time

-- User 2 borrows items
INSERT INTO Borrows (user_id, item_id, borrow_date, due_date, return_date, fine) VALUES
(2, 3, '2023-10-02', '2023-10-16', '2023-10-16', NULL),  -- Returned on time
(2, 4, '2023-10-06', '2023-10-20', NULL, NULL),          -- Still borrowed
(2, 5, '2023-10-08', '2023-10-22', '2023-10-23', 3),     -- Returned late, $3 fine
(2, 8, '2023-10-11', '2023-10-25', NULL, NULL),          -- Still borrowed
(2, 10, '2023-10-14', '2023-10-28', '2023-10-27', NULL); -- Returned on time

Running query in 'sqlite:///library.db'

5 rows affected.

5 rows affected.

++
||
++
++

In [83]:
%%sql
INSERT INTO RecommendedFor (event_id, audience_id) VALUES (10, 3);

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [87]:
%%sql 
DELETE FROM Attending;

Running query in 'sqlite:///library.db'

RuntimeError: (sqlite3.OperationalError) database is locked
[SQL: DELETE FROM Attending;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%%sql